In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
from keras.preprocessing.image import ImageDataGenerator
print(os.listdir("../input"))

# load and prepare data

In [12]:
img1=plt.imread("../input/automatic-classification-of-fetal-ultrasound-brain/data/data/Patient00216_Plane3_4_of_5.png")
"""img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
omg=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret,img=cv2.threshold(img,0,10,cv2.THRESH_BINARY)
plt.imshow(img,cmap="gray")"""
plt.imshow(img1,cmap="gray")

In [13]:
data=pd.read_csv("../input/automatic-classification-of-fetal-ultrasound-brain/train.csv")

In [14]:
path='../input/automatic-classification-of-fetal-ultrasound-brain/data/data/'

In [28]:
def load_image(i):
    d=plt.imread(path+str(i)+'.png')
    d.resize((224,224,1))
    return d
    

In [29]:
Y=[]
for i in data["class"]:
    Y.append(i)

In [30]:
X=[]
for i in data['image_name']:
    
    X.append(np.array(load_image(i)))

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
from tensorflow.keras.utils import to_categorical

In [33]:
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=to_categorical(Y,4)
X = np.array(X)


In [34]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

# load and use segmentation model

In [35]:
! git clone https://github.com/haochen23/foetal-head-segmentation Folder_name

In [36]:
UNET_PATH = './Folder_name/best_model_224_res.h5'
unet_model = tf.keras.models.load_model(UNET_PATH, compile=False)

In [37]:
unet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [38]:
Xtrain=unet_model.predict(x_train)
Xtest=unet_model.predict(x_test)

# model

In [39]:

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

In [40]:


model = Sequential()

model.add(Conv2D(filters = 256, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (224,224,1)))
model.add(Conv2D(filters = 256, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(4, activation = "sigmoid"))



In [41]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [42]:
epochs=2
batch_size=10
History =model.fit(Xtrain,y_train, batch_size=batch_size,
                              epochs = epochs, validation_data = (Xtest,y_test),
                              verbose = 1)

# submission

In [43]:
df=pd.read_csv("../input/automatic-classification-of-fetal-ultrasound-brain/test.csv")


In [44]:
X=[]
for i in df['image_name']:
    
    X.append(np.array(load_image(i)))

In [45]:
X=np.array(X)

In [46]:
results=model.predict(X)

In [47]:
pred=np.argmax(results,axis=1)

In [48]:
prediction=le.inverse_transform(pred)

In [49]:
sub=pd.read_csv("../input/automatic-classification-of-fetal-ultrasound-brain/sample_submission.csv")

In [52]:
sub['class']=prediction

In [53]:
sub.to_csv('sub.csv',index=False)